In [356]:
import numpy as np # for Linear algebra
import pandas as pd # for data manipulation/CSV I/O
pd.options.mode.chained_assignment = None    # To avoid the SettingWithCopyWarning

In [357]:
#Loading the dataset
deliveries=pd.read_csv("dataset/deliveries.csv")
matches =pd.read_csv("dataset/matches.csv")

In [358]:
#deliveries.head(1)

In [359]:
#matches.head()

In [360]:
#deliveries.columns

In [361]:
deliveries = deliveries[deliveries['wide_runs'] == 0]
#deliveries = deliveries[deliveries['is_super_over'] == 0]
#deliveries

In [362]:
#Keeping Only the required Columns
batsman = deliveries[deliveries['wide_runs'] == 0]
batsman = deliveries[deliveries['is_super_over'] == 0]
#batsman = batsman[batsman['noball_runs'] == 0]
#column_heads = ['match_id','batsman','batsman_runs']
column_heads = ['match_id','batsman','batsman_runs','ball']
batsman = batsman[column_heads]
#batsman = batsman[batsman['wide_runs'] == 0]
recent_bat = batsman.copy()
#batsman.head()

In [363]:
batsman["Fours"] = np.where(batsman["batsman_runs"] == 4, 1, 0)
batsman["Sixes"] = np.where(batsman["batsman_runs"] == 6, 1, 0)
balls = batsman.copy()
balls = balls.groupby(['match_id','batsman']).agg({'ball':'count','batsman_runs':'sum','Fours':'sum',
                                                  'Sixes':'sum'}).reset_index()
balls = balls.groupby(['batsman']).agg({'ball':'sum', 'batsman_runs':'sum','Fours':'sum',
                                       'Sixes':'sum'}).reset_index()
#balls[balls['batsman'] == 'Yuvraj Singh']
#balls.head(17)

In [364]:
batsmen_info = batsman.groupby(['batsman']).agg(
    {'match_id': lambda x: x.nunique(), 'batsman_runs': 'sum', 'Fours': 'sum', 'Sixes': 'sum'}).reset_index()

batter = batsman.groupby(['match_id','batsman'])
#batter.head()

In [365]:
#Calculate HS 30s 50s 
#temp = deliveries.groupby(['match_id','batsman'])

temp = batsman.groupby(['match_id','batsman'])
hs = temp.sum().reset_index().groupby('batsman').agg({'batsman_runs': 'max'}).reset_index()
thirty = temp.sum().reset_index().groupby(['match_id', 'batsman']).agg(
    {'batsman_runs': lambda x: 1 if (x > 29).any() else 0}).reset_index().groupby('batsman').agg(
    {'batsman_runs': 'sum'}).reset_index()

fifty = temp.sum().reset_index().groupby(['match_id', 'batsman']).agg(
    {'batsman_runs': lambda x: 1 if (x > 49).any() else 0}).reset_index().groupby('batsman').agg(
    {'batsman_runs': 'sum'}).reset_index()

tmp = deliveries.groupby(['match_id','batsman'])
balls = tmp.count().reset_index().groupby('batsman').agg({'batsman_runs': 'sum'}).reset_index()
#balls = temp.count().reset_index().groupby('batsman').agg({'ball': 'count'}).reset_index()

thirty = thirty.rename(columns={'batsman_runs': "30s"})
hs = hs.rename(columns={'batsman_runs': "HS"})
balls = balls.rename(columns={'batsman_runs': "Balls_faced"})
fifty = fifty.rename(columns={'batsman_runs' : '50s' })

batsmen_info = pd.merge(batsmen_info, hs, on=['batsman'])
batsmen_info = pd.merge(batsmen_info, thirty, on=['batsman'])
batsmen_info = pd.merge(batsmen_info, balls, on=['batsman'])
batsmen_info = pd.merge(batsmen_info, fifty, on=['batsman'])


In [366]:
#Calculating Strike Rate SR
batsmen_info['SR'] = np.around((batsmen_info['batsman_runs'] / batsmen_info['Balls_faced']) * 100, 2)
#print(batsmen_info.head())

In [367]:
#Arrange it in a Sequence
batsmen_info = batsmen_info.rename(columns={'batsman': 'Batsman', 'batsman_runs': 'Runs', 'match_id': 'Matches', 
                                            'Balls_faced' : 'Balls'})


In [368]:
#batsmen_info.sort_values(by='Sixes',ascending = False).reset_index(drop='T')[:10]

In [369]:
#Calculating Not Out and Average
dele = deliveries[deliveries['wide_runs'] == 0]
batsmen = dele.groupby(["match_id", "inning", "batting_team", "batsman"])
batsmen = batsmen["batsman_runs"].sum().reset_index()
#batsmen = batsmen['ball'].count()
not_out = deliveries[pd.notnull(deliveries['player_dismissed'])]
not_out = not_out[['match_id','inning','player_dismissed','bowler','dismissal_kind','fielder']]
not_out.rename(columns={'player_dismissed':'batsman'}, inplace = True)
batsmen=batsmen.merge(not_out,left_on=['match_id','inning','batsman'],right_on=['match_id','inning','batsman'], how= "left")
batsmen['dismissal_kind']=batsmen.dismissal_kind.fillna('not_out')
batsmen['fielder']=batsmen.fielder.fillna('-')

#print(batsmen.head())

dismissal_group=batsmen[['dismissal_kind','batsman']]
dismissal_group=dismissal_group.groupby(['batsman','dismissal_kind']).size().reset_index()
dismissal_group.rename(columns={'0':'No_of_times', 'batsman':'Batsman'},inplace=True)
not_outs=dismissal_group[dismissal_group['dismissal_kind']=='not_out']
not_outs=not_outs.drop(['dismissal_kind'],1)
not_outs.rename(columns={0:'NO'},inplace=True)
#print (not_outs.head())

batsmen_info = pd.merge(batsmen_info, not_outs, on=['Batsman'], how='left')
batsmen_info = batsmen_info.fillna(0)
#batsmen_info = pd.merge(batsmen_info, not_outs, on=['Batsman'])
#print(batsmen_info.head())

In [370]:
#Calculating Batsmen Average
#batsmen_info['Average'] = batsmen_info['Runs'] / (batsmen_info['Matches'] - batsmen_info['NO'])
batsmen_info['Average'] = np.where((batsmen_info['Runs'] / (batsmen_info['Matches'] - batsmen_info['NO'])) == np.inf,
                                   batsmen_info['Runs'] , 
                                   batsmen_info['Runs'] / (batsmen_info['Matches'] - batsmen_info['NO']))
batsmen_info = batsmen_info.replace([np.inf, -np.inf],np.nan)
#print(batsmen_info.head())
#batsmen_info.sort_values(by='Batsman', ascending=False).reset_index(drop='T')[:10]

In [371]:
#Calculating the recent Form of Players
req_coulms = ['batsman','batsman_runs','dismissal_kind']
recent_form = batsmen[req_columns]
recent_form['dismissal'] = np.where(batsmen['dismissal_kind'] == 'not_out' , 0, 1)
recent_form = recent_form.groupby('batsman').tail(4)
recent_form.drop('dismissal_kind',axis=1,inplace=True)
recent_form = recent_form.groupby(['batsman']).agg({'batsman_runs' : 'sum', 'dismissal' : 'sum'}).reset_index()
recent_form.rename(columns={'batsman':'Batsman','batsman_runs':'Recent_runs'}, inplace=True)
recent_form['Form_Average'] = np.where((recent_form['Recent_runs'] / recent_form['dismissal']) == np.inf,
                                       recent_form['Recent_runs'],
                                       recent_form['Recent_runs'] / recent_form['dismissal'])
recent_form.drop('dismissal',axis=1,inplace=True)
batsmen_info = pd.merge(batsmen_info, recent_form, on=['Batsman'])
#recent_form.head(5)

In [377]:
batsmen_info[batsmen_info['Batsman'] == 'RG Sharma']

,index,Batsman,Matches,Runs,Average,NO,Balls,SR,30s,50s,HS,Sixes,Fours,Recent_runs,Form_Average
347,347,RG Sharma,167,4496,32.345324,28.0,3429,131.12,62,35,109,185,379,56,14.0


In [373]:
sequence = ['Batsman', 'Matches', 'Runs','Average','NO', 'Balls' , 'SR', '30s', '50s' , 'HS', 'Sixes', 'Fours', 'Recent_runs',
           'Form_Average']
batsmen_info = batsmen_info[sequence].reset_index()
#print(batsmen_info.head())

In [374]:
export_csv = batsmen_info.to_csv ('dataset/batsmen.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path

In [375]:
batsmen_info[batsmen_info['Batsman'] == 'V Kohli']

,index,Batsman,Matches,Runs,Average,NO,Balls,SR,30s,50s,HS,Sixes,Fours,Recent_runs,Form_Average
458,458,V Kohli,155,4953,38.695312,27.0,3786,130.82,72,39,113,178,435,136,45.333333
